In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
import string

from nltk.tokenize import TweetTokenizer # doesn't split at apostrophes
import nltk
from nltk import Text
from nltk.tokenize import regexp_tokenize
from nltk.tokenize import word_tokenize  
from nltk.tokenize import sent_tokenize 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag


from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
#I CLEANED THE 9 TRAIN FILES, I RUN THESE 9 TIME
#change the NEW NAME and the PATH each time **********IMPORTANT

# BTW TRAIN 9 file is EMPTY
new_name="train1.csv";

#LOAD DATA
train = pd.read_csv("data/train1_cleaned.csv");
#train = train.drop('accuracy', 1)
#train = train.drop('views', 1)
#train = train.drop('licenseID', 1)
#train = train.drop('DateUploaded', 1)


#CLEAN DATA
for comments in train:
    # remove '\\n'
    train['photoTags'] = train['photoTags'].map(lambda x: re.sub('\\n',' ',str(x)))
    
    # remove any text starting with User... 
    train['photoTags'] = train['photoTags'].map(lambda x: re.sub("\[\[User.*",'',str(x)))
    
    # remove IP addresses or user IDs
    train['photoTags'] = train['photoTags'].map(lambda x: re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}",'',str(x)))
    
    # lower uppercase letters
    train['photoTags'] = train['photoTags'].map(lambda x: str(x).lower())
    
    #remove http links in the text
    train['photoTags'] = train['photoTags'].map(lambda x: re.sub("(http://.*?\s)|(http://.*)",'',str(x)))
    
    #remove all punctuation 
    train['photoTags'] = train['photoTags'].map(lambda x: re.sub("["+string.punctuation+"]",'',str(x)))
       # s= re.sub("["+string.punctuation+"]", " ", s)
    train['photoTags'] = train['photoTags'].map(lambda x: re.sub(r'[^\x00-\x7f]','',str(x)))
     #remove all 
    train['photoTags'] = train['photoTags'].map(lambda x: re.sub("geotagged",'',str(x)))
    train['photoTags'] = train['photoTags'].map(lambda x: re.sub("mm",'',str(x)))
    train['photoTags'] = train['photoTags'].map(lambda x: re.sub("mmf",'',str(x)))
    train['photoTags'] = train['photoTags'].map(lambda x: re.sub("geostate",'',str(x)))
    train['photoTags'] = train['photoTags'].map(lambda x: re.sub("landscape",'',str(x)))
    train['photoTags'] = train['photoTags'].map(lambda x: re.sub("flickrhq",'',str(x)))
    train['photoTags'] = train['photoTags'].map(lambda x: re.sub("admin",'',str(x)))
    train['photoTags'] = train['photoTags'].map(lambda x: re.sub("exz",'',str(x)))
    train['photoTags'] = train['photoTags'].map(lambda x: re.sub("msh",'',str(x)))
    train['photoTags'] = train['photoTags'].map(lambda x: re.sub("exdf",'',str(x)))
    train['photoTags'] = train['photoTags'].map(lambda x: re.sub("sigma",'',str(x)))
    train['photoTags'] = train['photoTags'].map(lambda x: re.sub("pentaxk",'',str(x)))
    train['photoTags'] = train['photoTags'].map(lambda x: re.sub("photoidimg",'',str(x)))
   
    #delete all rows with nulls, this works fine because only the photo tags has nulls
    train['photoTags'].replace('', np.nan, inplace=True)
    train['photoTags'].replace(' ', np.nan, inplace=True)
    train['photoTags'].replace('  ', np.nan, inplace=True)
    train['photoTags'].replace('   ', np.nan, inplace=True)
    train['photoTags'].replace('    ', np.nan, inplace=True)
    train['photoTags'].replace('     ', np.nan, inplace=True)
    train = train.dropna(how='any',axis=0) 
    train.to_csv(new_name, encoding="utf-8",index=False)

In [55]:
#df.isnull().sum()
df=train

In [ ]:
#Combine the 9 CSV files

fout=open("train_cleaned_all.csv","a")
# first file:
for line in open("train1_cleaned.csv"):
    fout.write(line)
# now the rest:    
for num in range(2,10):
    f = open("train"+str(num)+".csv", "r+")
    f.readline()# skip the header
    for line in f:
         fout.write(line)
    f.close() # not really needed
fout.close()

In [9]:
#CLEANING TEST
df=pd.read_csv("test.csv", encoding='latin-1');
df.head()

,photoID,latitude,longitude,accuracy,userID,photoLink,photoTags,DateTaken,DateUploaded,views,licenseID
0,1472189240,38.659015999999994,-57.767097,16,31911001@N00,http://farm2.staticflickr.com/1437/1472189240_...,fortune pei,1191331687,1191325386,18,1
1,1441071302,47.52434,-52.622547,16,12739532@N08,http://farm2.staticflickr.com/1070/1441071302_...,chimney rust stack cape spear,1188322363,1190776582,53,3
2,1438580776,47.57131,-52.680965,16,12739532@N08,http://farm2.staticflickr.com/1416/1438580776_...,sunset car newfoundland lights stjohns signalh...,1182804425,1190732692,126,3
3,156031551,64.09141,-72.1582,16,94366076@N00,http://farm1.staticflickr.com/62/156031551_8e6...,wifi broadband,1148959298,1148952328,32,2
4,168780851,-31.637306,129.43114,16,73115147@N00,http://farm1.staticflickr.com/78/168780851_f44...,geotagged coast australia cliffs sa southaustr...,1150510518,1150535718,174,3


In [10]:
df.isnull().sum()

photoID             0
latitude            0
longitude           0
accuracy            0
userID              0
photoLink           0
photoTags       35053
DateTaken           0
DateUploaded        0
views               0
licenseID           0
dtype: int64

In [11]:
df.count()

photoID         262009
latitude        262009
longitude       262009
accuracy        262009
userID          262009
photoLink       262009
photoTags       226956
DateTaken       262009
DateUploaded    262009
views           262009
licenseID       262009
dtype: int64

In [12]:
#delete all rows with nulls, this works fine because only the photo tags has nulls
df = df.dropna(how='any',axis=0) 
df.to_csv("test_cleaned.csv", encoding="utf-8",index=False)

In [18]:
df.isnull().sum()

photoID         0
latitude        0
longitude       0
accuracy        0
userID          0
photoLink       0
photoTags       0
DateTaken       0
DateUploaded    0
views           0
licenseID       0
dtype: int64

In [19]:
df.count()

photoID         226956
latitude        226956
longitude       226956
accuracy        226956
userID          226956
photoLink       226956
photoTags       226956
DateTaken       226956
DateUploaded    226956
views           226956
licenseID       226956
dtype: int64

In [17]:
#CLEAN DATA
for comments in df:
    # remove '\\n'
    df['photoTags'] = df['photoTags'].map(lambda x: re.sub('\\n',' ',str(x)))
    
    # remove any text starting with User... 
    df['photoTags'] = df['photoTags'].map(lambda x: re.sub("\[\[User.*",'',str(x)))
    
    # remove IP addresses or user IDs
    df['photoTags'] = df['photoTags'].map(lambda x: re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}",'',str(x)))
    
    # lower uppercase letters
    df['photoTags'] = df['photoTags'].map(lambda x: str(x).lower())
    
    #remove http links in the text
    df['photoTags'] = df['photoTags'].map(lambda x: re.sub("(http://.*?\s)|(http://.*)",'',str(x)))
    
    #remove all punctuation 
    df['photoTags'] = df['photoTags'].map(lambda x: re.sub("["+string.punctuation+"]",'',str(x)))
       # s= re.sub("["+string.punctuation+"]", " ", s)
    df['photoTags'] = df['photoTags'].map(lambda x: re.sub(r'[^\x00-\x7f]','',str(x)))
    
     # remove any text starting with User... 
    df['photoTags'] = df['photoTags'].map(lambda x: re.sub("\[\[User.*",'',str(x)))
 
#SAVE THE CLEANED DATA
df.to_csv("test_clean1.csv", encoding="utf-8",index=False) 